# AIG230 NLP (Week 3 Lab) — Notebook 2: Statistical Language Models (Train, Test, Evaluate)

This notebook focuses on **n-gram Statistical Language Models (SLMs)**:
- Train **unigram**, **bigram**, **trigram** models
- Handle **OOV** with `<UNK>`
- Apply **smoothing** (Add-k)
- Evaluate with **cross-entropy** and **perplexity**
- Do **next-word prediction** and simple **text generation**

> Industry framing: even if modern systems use neural LMs, n-gram LMs are still useful for
baselines, constrained domains, and for understanding evaluation.


### What is smoothing?

Smoothing is a way to stop a language model from saying “this can never happen.”

When we train a language model from data, it only knows what it has seen before.
If it never saw a particular word sequence, the model would normally give it a probability of zero.

Smoothing fixes that.
### Why is this a problem without smoothing?

Imagine the model learned English only by reading a small number of news articles.

If it never saw:

- “oil prices explode”

the model would conclude:

- “That sentence is impossible.”

But as humans, we know it could happen. The model just hasn’t seen it yet.

Without smoothing:

- One unseen word makes the whole sentence probability zero

- Evaluation breaks

- The model is too confident and too brittle

## 0) Setup


In [2]:

import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


## 1) Data: domain text you might see in real systems


We use short texts that resemble:
- release notes
- incident summaries
- operational runbooks
- customer support messaging

In practice, you would load thousands to millions of lines.


In [3]:

corpus = [
    "vpn disconnects frequently after windows update",
    "password reset link expired user cannot login",
    "api requests timeout when latency spikes",
    "portal returns 500 error after deployment",
    "email delivery delayed messages queued",
    "mfa prompt never arrives user stuck at login",
    "wifi drops in meeting rooms access point reboot helps",
    "outlook search not returning results index corrupted",
    "printer driver install fails with error 1603",
    "teams calls choppy audio jitter high",
    "permission denied accessing shared drive though in correct group",
    "battery drains fast after bios update power settings unchanged",
    "push notifications not working on android app",
    "mailbox full cannot receive emails auto archive not running",
]

# Train/test split at sentence level
random.seed(42)
random.shuffle(corpus)
split = int(0.75 * len(corpus))
train_texts = corpus[:split]
test_texts = corpus[split:]

len(train_texts), len(test_texts), train_texts[:2], test_texts[:2]


(10,
 4,
 ['printer driver install fails with error 1603',
  'push notifications not working on android app'],
 ['email delivery delayed messages queued',
  'vpn disconnects frequently after windows update'])

## 2) Tokenization + special tokens


We will:
- lowercase
- keep alphanumerics
- split on whitespace
- add sentence boundary tokens: `<s>` and `</s>`

We will also map rare tokens to `<UNK>` based on training frequency.


In [4]:

def tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

def add_boundaries(tokens: List[str], n: int) -> List[str]:
    # For n-grams, prepend (n-1) start tokens for simpler context handling
    return ["<s>"]*(n-1) + tokens + ["</s>"]

# Example
tokens = tokenize("Printer driver install fails with error 1603")
add_boundaries(tokens, n=3)


['<s>',
 '<s>',
 'printer',
 'driver',
 'install',
 'fails',
 'with',
 'error',
 '1603',
 '</s>']

## 3) Build vocabulary and handle OOV with <UNK>


In [5]:

# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

freq = Counter(train_tokens_flat)

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = 2
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)


(['email', 'delivery', 'delayed', 'messages', 'queued'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>'])

## 4) Train n-gram counts (unigram, bigram, trigram)


We will compute:
- `ngram_counts[(w1,...,wn)]`
- `context_counts[(w1,...,w_{n-1})]`

Then probability:
\ndefault:  P(w_n | context) = count(context + w_n) / count(context)

This fails when an n-gram is unseen, so we add smoothing.


In [6]:
def get_ngrams(tokens: List[str], n:int) -> List[Tuple[str, ...]]:
  return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def train_ngram_counts(texts: List[str], n:int) -> Dict[Tuple[str, ...], int]:
  ngrams_counts = Counter()
  context_counts = Counter()
  for text in texts:
    toks = replace_oov(tokenize(text), vocab)
    toks = add_boundaries(toks, n)
    for ng in get_ngrams(toks, n):
      ngrams_counts[ng] += 1
      context = ng[:-1]
      context_counts[context] += 1
  return ngrams_counts, context_counts


In [7]:
unigram_counts, unigram_context = train_ngram_counts(train_texts, n=1, vocab=vocab )
unigram_counts

TypeError: train_ngram_counts() got an unexpected keyword argument 'vocab'

## 5) Add-k smoothing and probability function


### What does Add-k smoothing do?
Add-k smoothing tells the model:

- “Even if you didn’t see something, assume it could still happen a little bit.”

It does this by:

- Giving every possible next word a tiny amount of probability

- Not just the ones seen in training

So instead of:

- seen → possible

- unseen → impossible

We get:

- seen → more likely

- unseen → less likely, but still possible


### Why is it called Add-k?

Because we add a small number k to every word count.

Think of it as:

- adding a tiny “imaginary observation” for every word

- so no word ever has zero probability

When k is small (like 0.1 or 0.5), it gently smooths the probabilities instead of overpowering real data.

In [ ]:
# This function calculates the probability of a word appearing next, given the previous words, while making sure the probability is never zero.
def prob_addk(ngram: Tuple[str, ...], ngram_counts: Counter, context_counts: Counter, V:int, k: float = 0.5) -> float:
  """
  Computer add-k P(w_n | w_1 ... w_{n-1})
  where ngram = (w_1 ... w_{n-1})
  """
  context = ngram[:-1]
  # formula for add-k explained in the slides
  return  (ngram_counts[ngram] + k) / (context_counts[context] + k * V)
  

In [ ]:
V = len(vocab)
example = ("<s>", "I", "love", "AI")
prob_addk(example, bigram_counts, bigram_context, V, k=0.5)

NameError: name 'bigram_counts' is not defined

## 6) Evaluate: cross-entropy and perplexity on test set


We evaluate an LM by how well it predicts held-out text.

Cross-entropy (average negative log probability):
H = - (1/N) * sum log2 P(w_i | context)

Perplexity:
PP = 2^H

Lower perplexity is better.


In [8]:
def evaluate_perplexity(texts: List[str], n: int, ngram_counts: Counter, 
                        context_counts: Counter, vocab: set, k: float = 0.5) -> float:
    """
    Calculate perplexity on test texts.
    Lower perplexity = model is less "surprised" = better predictions.
    """
    V = len(vocab)
    total_log_prob = 0.0
    total_tokens = 0
    
    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        
        for ng in get_ngrams(toks, n):
            prob = prob_addk(ng, ngram_counts, context_counts, V, k)
            total_log_prob += math.log2(prob)
            total_tokens += 1
    
    # Cross-entropy H = negative average log probability
    H = -total_log_prob / total_tokens
    # Perplexity = 2^H
    return 2 ** H

# Test it
V = len(vocab)
print("Bigram Perplexity:", evaluate_perplexity(test_texts, 2, bigram_counts, bigram_context, vocab))

NameError: name 'bigram_counts' is not defined

## 7) Next-word prediction (top-k)


Given a context, compute the probability of each candidate next token and return the top-k.

This mirrors:
- autocomplete in constrained domains
- template suggestion systems
- command prediction in runbooks


In [9]:
def predict_next_word(context: str, n: int, ngram_counts: Counter, 
                      context_counts: Counter, vocab: set, top_k: int = 5, k: float = 0.5):
    """Given a context string, predict the top-k most likely next words."""
    V = len(vocab)
    
    # Tokenize and prepare context
    context_toks = replace_oov(tokenize(context), vocab)
    
    # Get last (n-1) tokens for context
    if n > 1 and len(context_toks) >= n-1:
        ctx = tuple(context_toks[-(n-1):])
    elif n > 1:
        ctx = tuple(["<s>"] * (n-1 - len(context_toks)) + context_toks)
    else:
        ctx = tuple()
    
    # Score each possible next word
    predictions = []
    for word in vocab:
        if word in ["<s>", "</s>", "<UNK>"]:
            continue
        ng = ctx + (word,)
        prob = prob_addk(ng, ngram_counts, context_counts, V, k)
        predictions.append((word, prob))
    
    # Sort by probability, return top-k
    predictions.sort(key=lambda x: -x[1])
    return predictions[:top_k]

# Test it
print("Top predictions for 'vpn':")
for word, prob in predict_next_word("vpn", 2, bigram_counts, bigram_context, vocab):
    print(f"  {word}: {prob:.4f}")

Top predictions for 'vpn':


NameError: name 'bigram_counts' is not defined

## 8) Simple generation (bigram or trigram)


Text generation is not the main goal in SLMs, but it helps you verify:
- boundary handling
- smoothing
- OOV decisions

We will sample tokens until we hit `</s>`.


In [ ]:

def sample_next(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5):
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    words = [w for w in vocab if w != "<s>"]
    probs = []
    for w in words:
        ng = context + (w,)
        probs.append(prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth))
    # Normalize
    s = sum(probs)
    probs = [p/s for p in probs]
    return random.choices(words, weights=probs, k=1)[0]

def generate(n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, max_len: int = 20, k_smooth: float = 0.5):
    tokens = ["<s>"]*(n-1) if n > 1 else []
    out = []
    for _ in range(max_len):
        w = sample_next(tokens, n, ngram_counts, context_counts, vocab, k_smooth=k_smooth)
        if w == "</s>":
            break
        out.append(w)
        tokens.append(w)
    return " ".join(out)

for _ in range(5):
    print("BIGRAM:", generate(2, bi_counts, bi_ctx, vocab, max_len=18))


## 9) Model comparison: effect of n and smoothing


Try different `k` values. Notes:
- `k=1.0` is Laplace smoothing (often too strong)
- smaller `k` (like 0.1 to 0.5) is often better

In real corpora, trigrams often beat bigrams, but require more data.


In [10]:

# Compare different smoothing values
print("Effect of smoothing parameter k on perplexity:")
print("-" * 40)
for k_val in [0.1, 0.5, 1.0, 2.0]:
    ppl = evaluate_perplexity(test_texts, 2, bigram_counts, bigram_context, vocab, k=k_val)
    print(f"k = {k_val}: Perplexity = {ppl:.2f}")

print("\nCompare n-gram orders:")
print("-" * 40)
print(f"Unigram: {evaluate_perplexity(test_texts, 1, unigram_counts, unigram_context, vocab):.2f}")
print(f"Bigram:  {evaluate_perplexity(test_texts, 2, bigram_counts, bigram_context, vocab):.2f}")
print(f"Trigram: {evaluate_perplexity(test_texts, 3, trigram_counts, trigram_context, vocab):.2f}")

Effect of smoothing parameter k on perplexity:
----------------------------------------


NameError: name 'bigram_counts' is not defined

## Exercises (do these during lab)
1) Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.  
2) Change `min_count` (OOV threshold) and explain how perplexity changes.  
3) Implement **backoff**: if a trigram is unseen, fall back to bigram; if unseen, fall back to unigram.
